<h1> Counting Summations </h1>

It is possible to write $5$ as a sum in exactly $6$ different ways:
\begin{align*}
    &4 + 1 \\
    &3 + 2 \\
    &3 + 1 + 1 \\
    &2 + 2 + 1 \\
    &2 + 1 + 1 + 1 \\
    &1 + 1 + 1 + 1 + 1 
\end{align*}

How many different ways can $100$ be written as a sum of at least $2$ positive integers?

---

The partition of integers can be described by the recurrence relation,
\begin{gather}
    p_n = \sum_{k=1}^{n} p_k(n) \\
    p_k(n) = p_k(n-k) + p_{k-1}(n-1) \\
    p(n) = \sum_{k \in \mathcal{z}} (-1)^{k+1} p(n-k(3k-1)/2)
\end{gather}
<!-- https://www.whitman.edu/mathematics/cgt_online/book/section03.03.html -->
<!-- https://en.wikipedia.org/wiki/Partition_function_(number_theory) -->

In [2]:
// use std::ops::{Add, Mul, Div};

// extern crate num;
// use num::PrimInt;
// use num::cast::ToPrimitive;

In [3]:

// fn sum_first_n_natural_numbers<T> (n:T) -> T 
//     where T: Add<Output = T> + Mul<Output = T> + Div<Output = T> + PrimInt
// {
//     let two = num::cast(2).unwrap();
//     n*(n + T::one())/two
// }

In [4]:
// fn number_of_summations<T>(n: T) -> T 
//     where T: Mul<Output = T> + Div<Output = T> + PrimInt + std::fmt::Debug
// {
//     if n == num::cast(2).unwrap() {
//         return T::one()
//     }
    
//     let is_even : bool = n & num::one() == num::zero();
//     let two: T = num::cast(2).unwrap();
//     let half_n = n/two;
    
// //     println!("{} {:?}", is_even, half_n);
    
//     if is_even {
//         two*sum_first_n_natural_numbers(half_n) - half_n
//     } else { 
//         two*sum_first_n_natural_numbers(half_n)  
//     }
    
// }

## Brute force

In [5]:
(1..10).for_each(|x| println!("{} {}",x*(3*x-1)/2, x*(3*x-1)/2+x));

1 2
5 7
12 15
22 26
35 40
51 57
70 77


In [6]:
fn recurrent_partition_function(input_num: i32) -> i32 
{
    // Put lowest cases into match to reduce computational time
    match input_num {
        0 | 1 => return 1,
        2 => return 2,
        3 => return 3,
        4 => return 5,
        5 => return 7,
        6 => return 11,
        num if num < 0 => return 0,
        _ => ()
    };
    
    let mut sum = 0;
    let mut k = 1;
    
    loop {
        let x: i32 = (k*(3*k - 1))/2;
        
        if x > input_num + k {
            return sum;
        }
        
        let y = recurrent_partition_function(input_num - x) + recurrent_partition_function(input_num - x - k);
        
        if (k+1) & 1 == 1 {
            sum -= y
        } else {
            sum += y
        }
        
        k += 1;
    }
}

92 100
117 126


In [7]:
let soln = recurrent_partition_function(40);
println!("p(100) = {}", soln);

p(100) = 37338


## A bit less brute force

In [8]:
fn find_integer_partition(input_num: u32) -> u64 {
    
    let store_partition_vals = vec![0_u64; 1 + input_num as usize];
    
    
    
    0
}

In [9]:
// use std::collections::HashSet;

// fn required_integers(input_num: u32) -> HashSet<u32> {
    
    
// //     if input_num == 1 {
// //         out.insert(1);
// //         return out;
// //     } else if input_num == 2{
// //         out.insert(1);
// //         out.insert(2);
// //         return out;
// //     }
    
//     if input_num == 1 || input_num == 0 {
//         return HashSet::new();
//     }
    
//     let mut out;
    
//     out = HashSet::with_capacity(input_num as usize);
    
// //     let out: HashSet<u32> = HashSet::with_capacity(input_num as usize);
//     let max_k = ((-1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as usize;
//     println!("input num = {} max k = {}", input_num, max_k);
    
//     if max_k < 2 {
//         out.insert(input_num - 1);
//         out.insert(input_num - 2);
//         return out;
//     }
    
//     for k in 1..=max_k {
//         let higher_val = ((3*k*k + k)/2) as u32;
//         let lower_val = ((3*k*k - k)/2) as u32;
        
//         if higher_val <= input_num {
//             out.insert(input_num - higher_val);
//             out.insert(input_num - lower_val);
//         } else if lower_val <= input_num {
//             out.insert(input_num - lower_val);
//         }
//         println!("k = {} set = {:?}",k, out);
        
//     };
    
    
//     let mut updated_vals: HashSet<u32> = out.clone();
    
//     for item in out.iter() {
//         if item > &8 {
// //             let child_vals: HashSet<u32> = required_integers(*item).union(&out).into_iter().map(|x| x.clone()).collect();
//             let child_vals: HashSet<u32> = required_integers(*item);
// //             out = out.union(&&child_vals).collect();
//             updated_vals = out.union(&child_vals).into_iter().map(|x| x.clone()).collect::<HashSet<u32>>();
//         }
// //         let child_vals = required_integers(*item);
// //         out.union(&child_vals);
//     }
    
    
// //         out.union(&required_integers(higher_val));
// //         out.union(&required_integers(lower_val));
//     updated_vals.shrink_to_fit();
    
//     updated_vals
// }

In [10]:
fn higher_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num + num)/2) as u32
}
fn lower_pentagonal_num(num: u32) -> u32 {
    return ((3*num*num - num)/2) as u32
}


In [11]:
fn maximum_k_val(input_num: u32) -> u32 {
    ((-1.0_f32 + (1.0 + 24_f32*input_num as f32).sqrt())/6.0) as u32
}

In [12]:
fn compute_pentagonal_num(max_k: u32) -> Vec<u32> {
    (1..=max_k).map(|k| higher_pentagonal_num(k))
                .into_iter()
                .chain(
                    (1..=max_k).map(|k| lower_pentagonal_num(k))
                    .into_iter())
                .collect()
}

In [13]:
use std::collections::HashSet;

fn required_integers(input_num: u32) -> HashSet<u32> {
    let mut out = HashSet::with_capacity(input_num as usize);
    
    let max_k = maximum_k_val(input_num);
    
    let pentagonal_nums = compute_pentagonal_num(max_k);
    
    for counter in 0..max_k {
        let current_pent_num = pentagonal_nums.get(counter as usize).unwrap();
        
        let mut inner_counter = 0;
        
        loop {
            let n_time_k = inner_counter*current_pent_num;
            if n_time_k >= input_num {
                break
            } else {
                out.insert(input_num - n_time_k);
                inner_counter += 1;
            }
        }
    }
    
    
    out
}

In [14]:
let a = required_integers(100);
println!("{:?}", a);
// let mut combined = a.clone();
// a.iter().for_each(|val| combined.union(&required_integers(*val)));
// println!("{:?}", a);

for k in 1..100{
    println!("{} {}", (3*k*k - k)/2, (3*k*k + k)/2)
}

{43, 36, 10, 82, 64, 4, 37, 65, 14, 20, 60, 72, 96, 88, 100, 51, 66, 6, 58, 12, 23, 70, 16, 48, 8, 92, 93, 9, 78, 52, 86, 62, 84, 90, 46, 2, 42, 74, 54, 55, 34, 32, 30, 25, 68, 22, 38, 76, 56, 98, 26, 85, 40, 28, 80, 94, 50, 44, 24, 18, 79}
1 2
5 7
12 15
22 26
35 40
51 57
70 77
92 100
117 126
145 155
176 187
210 222
247 260
287 301
330 345
376 392
425 442
477 495
532 551
590 610
651 672
715 737
782 805
852 876
925 950
1001 1027
1080 1107
1162 1190
1247 1276
1335 1365
1426 1457
1520 1552
1617 1650
1717 1751
1820 1855
1926 1962
2035 2072
2147 2185
2262 2301
2380 2420
2501 2542
2625 2667
2752 2795
2882 2926
3015 3060
3151 3197
3290 3337
3432 3480
3577 3626
3725 3775
3876 3927
4030 4082
4187 4240
4347 4401
4510 4565
4676 4732
4845 4902
5017 5075
5192 5251
5370 5430
5551 5612
5735 5797
5922 5985
6112 6176
6305 6370
6501 6567
6700 6767
6902 6970
7107 7176
7315 7385
7526 7597
7740 7812
7957 8030
8177 8251
8400 8475
8626 8702
8855 8932
9087 9165
9322 9401
9560 9640
9801 9882
10045 10127
10292 

()

12105 12195
12376 12467
12650 12742
12927 13020
13207 13301
13490 13585
13776 13872
14065 14162
14357 14455
14652 14751
